In [1]:
!pip install gdown


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import cv2
from transformers import (
    XCLIPModel,
    XCLIPProcessor,
    SegformerForSemanticSegmentation,
    SegformerImageProcessor,
)
import numpy as np
import pandas as pd
from model import EmbeddingClassifier

import warnings
import gdown  # Добавляем импорт библиотеки gdown

# Игнорировать все предупреждения
warnings.filterwarnings("ignore")

# ======== Глобальные параметры ======== #
video_folder = r"C:\Users\pasha\OneDrive\Рабочий стол\test_dataset_rzhd_test\test РЖД ПДД\videos\videos"  # Папка с видео
output_csv = r'submission.csv'
MODEL_NAME = "microsoft/xclip-base-patch16"  # Не менять

# Путь к кастомной модели YOLO и ссылка для скачивания
YOLO_CUSTOM_PATH = r"best_93.pt"
YOLO_CUSTOM_URL = "https://drive.google.com/uc?id=1KovZgdLspcwOmxXVUmWcG3ro_TnrGT5n"

# Путь к модели SegFormer и ссылка для скачивания
SEGFORMER_MODEL_PATH = r"segformer_model"
SEGFORMER_MODEL_URL = "https://drive.google.com/drive/folders/1zrp_r4Iy1STrPYG9m0b96nuEiHLq7NNk"

BEST_MODEL_PATH = 'best_model_dataset_1_47.pth'  # Обученная модель классификатора

APPLY_PREPROCESSING = True  # Переключатель для применения предварительной обработки

# ======== Список меток и штрафов ======== #
LABEL_LIST = ['Статья 12.16. часть 1 Несоблюдение требований, предписанных дорожными знаками или разметкой проезжей части дороги',
                 'нарушений нет',
               'Статья 12.16 часть 2 Поворот налево или разворот в нарушение требований, предписанных дорожными знаками или разметкой проезжей части дороги',

                   'Статья 12.17  часть 1.1 и 1.2. движение транспортных средств по полосе для маршрутных транспортных средств или остановка на указанной полосе в нарушение Правил дорожного движения ',
                     'Статья 12.12 часть 2 1. невыполнение требования ПДД об остановке перед стоп-линией, обозначенной дорожными знаками или разметкой проезжей части дороги, при запрещающем сигнале светофора или запрещающем жесте регулировщика',
                       'Статья 12.15 часть 4 Выезд в нарушение правил дорожного движения на полосу, предназначенную для встречного движения, при объезде препятствия, либо на трамвайные пути встречного направления, за исключением случаев, предусмотренных частью 3 настоящей статьи']
NUM_CLASSES = len(LABEL_LIST)

FINE_DICT = {
    'нарушений нет': 0,
    'Статья 12.16. часть 1 Несоблюдение требований, предписанных дорожными знаками или разметкой проезжей части дороги': 500,
    'Статья 12.16 часть 2 Поворот налево или разворот в нарушение требований, предписанных дорожными знаками или разметкой проезжей части дороги': 1000,
    'Статья 12.17  часть 1.1 и 1.2. движение транспортных средств по полосе для маршрутных транспортных средств или остановка на указанной полосе в нарушение Правил дорожного движения ': 1500,
    'Статья 12.12 часть 2 1. невыполнение требования ПДД об остановке перед стоп-линией, обозначенной дорожными знаками или разметкой проезжей части дороги, при запрещающем сигнале светофора или запрещающем жесте регулировщика': 800,
    'Статья 12.15 часть 4 Выезд в нарушение правил дорожного движения на полосу, предназначенную для встречного движения, при объезде препятствия, либо на трамвайные пути встречного направления, за исключением случаев, предусмотренных частью 3 настоящей статьи': 5000
}

# ======== Функция для загрузки файлов ======== #
def download_file_if_not_exists(file_path, url, is_folder=False):
    if not os.path.exists(file_path):
        print(f"Файл {file_path} не найден. Начинаю загрузку...")
        if is_folder:
            # Для загрузки папки используем флаг --folder
            gdown.download_folder(url, output=file_path, quiet=False)
        else:
            gdown.download(url, output=file_path, quiet=False)
        print(f"Загрузка {file_path} завершена.")
    else:
        print(f"Файл {file_path} уже существует.")

# ======== Проверка и загрузка моделей ======== #
if APPLY_PREPROCESSING:
    # Проверяем и загружаем кастомную модель YOLO
    download_file_if_not_exists(YOLO_CUSTOM_PATH, YOLO_CUSTOM_URL)

    # Проверяем и загружаем модель SegFormer
    download_file_if_not_exists(SEGFORMER_MODEL_PATH, SEGFORMER_MODEL_URL, is_folder=True)

# ======== Загрузка моделей ======== #
# Устройство (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

# Загрузка XCLIP модели и процессора
processor = XCLIPProcessor.from_pretrained(MODEL_NAME)
model = XCLIPModel.from_pretrained(MODEL_NAME)
model.to(device)
model.eval()  # Переводим модель в режим оценки

# Загрузка модели классификатора и весов
classifier_model = EmbeddingClassifier(model.config.projection_dim, NUM_CLASSES)
classifier_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
classifier_model.to(device)
classifier_model.eval()

# Получение mean и std для нормализации изображений
try:
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std
except AttributeError:
    image_mean = processor.feature_extractor.image_mean
    image_std = processor.feature_extractor.image_std

# Определение видео трансформаций
video_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=image_mean, std=image_std)
])

# ======== Класс для предварительной обработки ======== #
class Preprocessor:
    def __init__(self, yolo_custom_path, segformer_model_path):
        self.device = device
        # Загрузка кастомной модели YOLOv5
        self.custom_model = torch.hub.load(
            'ultralytics/yolov5', 'custom', path=yolo_custom_path, force_reload=True
        ).to(self.device).eval()
        # Загрузка предобученной модели YOLOv5
        self.pretrained_model = torch.hub.load(
            'ultralytics/yolov5', 'yolov5n', pretrained=True
        ).to(self.device).eval()
        # Загрузка модели SegFormer
        self.segformer_model = SegformerForSemanticSegmentation.from_pretrained(
            segformer_model_path
        ).to(self.device).eval()
        self.extractor = SegformerImageProcessor()
        # Параметры
        self.traffic_related_classes = ["car", "bus", "truck", "motorcycle", "bicycle"]
        self.target_class_id = 2  # Целевой класс для SegFormer

    def apply(self, frame):
        height, width, _ = frame.shape

        # Преобразование кадра
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(rgb_frame)

        # Получение результатов от моделей YOLOv5
        results_pretrained = self.pretrained_model(img)
        results_custom = self.custom_model(img)

        # Объединение результатов
        results_combined = pd.concat(
            [results_pretrained.pandas().xyxy[0], results_custom.pandas().xyxy[0]],
            ignore_index=True,
        )

        # Обработка кадра моделью SegFormer
        seg_map = self.predict_segformer(rgb_frame)

        # Создание маски для затемнения
        mask = np.zeros((height, width), dtype=np.uint8)

        # Добавление результатов YOLOv5 в маску
        for _, row in results_combined.iterrows():
            if row["name"] in self.traffic_related_classes or row["confidence"] > 0.25:
                x1 = int(max(0, row["xmin"]))
                y1 = int(max(0, row["ymin"]))
                x2 = int(min(width - 1, row["xmax"]))
                y2 = int(min(height - 1, row["ymax"]))
                mask[y1:y2, x1:x2] = 255  # Область, которую не затемняем

        # Добавление результатов SegFormer в маску
        if seg_map.shape != (height, width):
            seg_map_resized = cv2.resize(seg_map, (width, height), interpolation=cv2.INTER_NEAREST)
        else:
            seg_map_resized = seg_map
        seg_mask = np.where(seg_map_resized == self.target_class_id, 255, 0).astype(np.uint8)
        mask = cv2.bitwise_or(mask, seg_mask)

        # Создание итогового кадра с затемнением
        alpha_mask = np.stack([mask, mask, mask], axis=-1)  # Создаем маску с 3 каналами
        frame_darkened = (frame * 0.2).astype(np.uint8)
        frame_result = np.where(alpha_mask == 255, frame, frame_darkened)

        return frame_result

    def predict_segformer(self, image):
        inputs = self.extractor(images=image, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.segformer_model(**inputs)
        logits = outputs.logits  # [batch_size, num_classes, height, width]
        segmentation = torch.argmax(logits, dim=1).squeeze(0)
        return segmentation.cpu().numpy()

# Инициализация препроцессора при необходимости
if APPLY_PREPROCESSING:
    preprocessor = Preprocessor(YOLO_CUSTOM_PATH, SEGFORMER_MODEL_PATH)
else:
    preprocessor = None

# ======== Функции для обработки видео ======== #
def extract_frames_from_video(
    video_capture, start_time, end_time, num_frames=8, preprocessor=None
):
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame

    frame_indices = np.linspace(start_frame, end_frame - 1, num=num_frames, dtype=int)
    frames = []

    for frame_idx in frame_indices:
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        success, frame = video_capture.read()
        if not success:
            break

        # Применение предварительной обработки
        if preprocessor is not None:
            frame = preprocessor.apply(frame)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)

        # Используем заданные трансформации
        frame_tensor = video_transform(frame_pil)
        frames.append(frame_tensor)

    # Повторяем последний кадр, если кадров меньше, чем num_frames
    while len(frames) < num_frames:
        frames.append(frames[-1].clone() if len(frames) > 0 else torch.zeros(3, 224, 224))

    # Преобразуем список тензоров в один тензор и перемещаем на устройство
    video_frames_tensor = torch.stack(frames).to(device)

    return video_frames_tensor

# ======== Функция для предсказания класса сегмента ======== #
def predict_segment_class(video_capture, start_time, end_time, preprocessor=None):
    # Извлекаем кадры из сегмента
    video_frames_tensor = extract_frames_from_video(
        video_capture, start_time, end_time, num_frames=8, preprocessor=preprocessor
    )
    video_frames_tensor = video_frames_tensor.unsqueeze(0)  # Добавляем размерность batch

    # Генерируем фиктивный текстовый ввод
    text_inputs = processor(
        text=[""],  # Пустой текст, так как мы используем только видеоэмбеддинги
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=77
    )
    input_ids = text_inputs['input_ids'].to(device)
    attention_mask = text_inputs['attention_mask'].to(device)

    # Получаем видеоэмбеддинги из модели XCLIP
    with torch.no_grad():
        outputs = model(
            pixel_values=video_frames_tensor,
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        video_embeds = outputs.video_embeds  # [batch_size, projection_dim]

    # Передаем эмбеддинги в классификатор
    with torch.no_grad():
        logits = classifier_model(video_embeds)
        probabilities = torch.softmax(logits, dim=1)
        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        print(predicted_class_idx)
        predicted_class = LABEL_LIST[predicted_class_idx]
        confidence = probabilities[0, predicted_class_idx].item()

    return predicted_class, confidence

# ======== Основная функция для обработки видео ======== #
def process_video(video_path, preprocessor=None):
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print(f"Не удалось открыть видео: {video_path}")
        return []

    fps = video_capture.get(cv2.CAP_PROP_FPS)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    print(f"Видео {os.path.basename(video_path)} длительностью {duration:.2f} секунд, FPS: {fps}")

    segment_duration = 10  # Продолжительность сегмента в секундах
    predictions = []

    num_segments = int(np.ceil(duration / segment_duration))
    for i in range(num_segments):
        start_time = i * segment_duration
        end_time = min((i + 1) * segment_duration, duration)

        predicted_class, confidence = predict_segment_class(
            video_capture, start_time, end_time, preprocessor=preprocessor
        )

        # Вычисляем среднее время сегмента
        violation_time = int((start_time + end_time) / 2)

        # Получаем сумму штрафа
        fine_amount = FINE_DICT.get(predicted_class, 0)

        # Если нарушение отсутствует, не добавляем его в предсказания
        if predicted_class == 'нарушений нет':
            continue

        print(f"Сегмент {i+1}/{num_segments}, время: {start_time:.2f}-{end_time:.2f} сек, "
              f"Класс: {predicted_class}, Доверие: {confidence:.4f}, Время нарушения: {violation_time} сек")

        predictions.append({
            'номер видео': os.path.splitext(os.path.basename(video_path))[0],
            'наименование нарушения': predicted_class,
            'сумма штрафа, руб.': fine_amount,
            'время нарушения (в секундах)': float(violation_time)
        })

    video_capture.release()
    return predictions

# ======== Обработка всех видео и формирование сабмита ======== #
def create_submission(video_paths, output_csv='submission.csv', preprocessor=None):
    all_predictions = []

    for video_path in video_paths:
        predictions = process_video(video_path, preprocessor=preprocessor)
        all_predictions.extend(predictions)

    submission_df = pd.DataFrame(all_predictions)
    submission_df.sort_values(by=['номер видео', 'время нарушения (в секундах)'], inplace=True)
    submission_df.to_csv(output_csv, index=False)
    print(f"Сабмит сохранен в файл {output_csv}")

# ======== Запуск скрипта ======== #
if __name__ == "__main__":
    # Список видеофайлов для обработки
    video_files = [
        os.path.join(video_folder, filename)
        for filename in os.listdir(video_folder)
        if filename.endswith(('.mp4', '.mov', '.avi'))  # Замените на нужные расширения файлов
    ]

    # Обработка видео и создание сабмита
    create_submission(video_files, output_csv=output_csv, preprocessor=preprocessor)


c:\Users\pasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Файл best_93.pt уже существует.
Файл segformer_model уже существует.
Используется устройство: cuda


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\pasha/.cache\torch\hub\master.zip
YOLOv5  2024-11-10 Python-3.11.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)

Fusing layers... 
YOLOv5s summary: 224 layers, 7167184 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\pasha/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-10 Python-3.11.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


Видео akn00005_0xsmsNYD.mov длительностью 300.03 секунд, FPS: 29.97002997002997
5
Сегмент 1/31, время: 0.00-10.00 сек, Класс: Статья 12.15 часть 4 Выезд в нарушение правил дорожного движения на полосу, предназначенную для встречного движения, при объезде препятствия, либо на трамвайные пути встречного направления, за исключением случаев, предусмотренных частью 3 настоящей статьи, Доверие: 0.3878, Время нарушения: 5 сек
0
Сегмент 2/31, время: 10.00-20.00 сек, Класс: Статья 12.16. часть 1 Несоблюдение требований, предписанных дорожными знаками или разметкой проезжей части дороги, Доверие: 0.5259, Время нарушения: 15 сек
1
Сегмент 3/31, время: 20.00-30.00 сек, Класс: Статья 12.16 часть 2 Поворот налево или разворот в нарушение требований, предписанных дорожными знаками или разметкой проезжей части дороги, Доверие: 0.3473, Время нарушения: 25 сек
1
Сегмент 4/31, время: 30.00-40.00 сек, Класс: Статья 12.16 часть 2 Поворот налево или разворот в нарушение требований, предписанных дорожными зн